In [103]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Bidirectional, Dropout, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import tensorflow as tf

In [2]:
# Sample data (replace with real data)
manual_abstracts = ["In the modern technology industry, the utilization of machine learning and natural language processing is increasingly dominant, with applications like facial recognition leveraging machine learning algorithms to understand and interpret users' natural language."]
keyword_queries = ["machine learning OR natural language processing"]

In [3]:
# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(manual_abstracts + keyword_queries)

In [4]:
max_length = 100  # Define a maximum length for padding sequences

In [5]:
def preprocess_text(text):
    tokens = tokenizer.texts_to_sequences([text])
    tokens_padded = pad_sequences(tokens, maxlen=max_length, padding='post')
    return tokens_padded

In [6]:
manual_abstracts_seq = np.array([preprocess_text(abstract)[0] for abstract in manual_abstracts])
keyword_queries_seq = np.array([preprocess_text(query)[0] for query in keyword_queries])

In [43]:
# Create the Siamese network
embedding_dim = 50
vocab_size = len(tokenizer.word_index) + 1
embedding_layer = Embedding(vocab_size, embedding_dim, input_length=max_length)

In [86]:
# Define the LSTM encoder
lstm_layer = LSTM(50)

In [87]:
# Define the input layers
input_abstract = Input(shape=(max_length,))
input_query = Input(shape=(max_length,))

In [88]:
# Encode the inputs
encoded_abstract = lstm_layer(embedding_layer(input_abstract))
encoded_query = lstm_layer(embedding_layer(input_query))

In [89]:
# Compute the similarity as the absolute difference between the encodings
similarity = Lambda(lambda x: K.abs(x[0] - x[1]))([encoded_abstract, encoded_query])
similarity = Dense(1, activation='sigmoid')(similarity)

In [99]:
# Define the model
model = Model(inputs=[input_abstract, input_query], outputs=similarity)

In [100]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust learning rate as needed

# Compile the model with the customized optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [101]:
# Dummy labels for training (replace with real labels)
labels = np.array([1])

In [102]:
# Train the model with the new labels
model.fit([manual_abstracts_seq, keyword_queries_seq], labels, batch_size=16, epochs=100)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.0043 - accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 102ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 82ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 81ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 96ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 81ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 89ms/step - loss: 0.6619 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 93ms/step - loss: 0.6617 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 106ms/step - loss: 0.6613 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 85ms/step - loss: 0.6610 - accuracy: 1.0000
Epoch 11/100
1/1 [=

In [94]:
# Save the model
model.save('similarity_model.h5')
print("Model saved as similarity_model.h5")

Model saved as similarity_model.h5


In [95]:
# Function to test similarity between two texts
def test_similarity(manual_abstract, keyword_query):
    manual_abstract_seq = preprocess_text(manual_abstract)
    keyword_query_seq = preprocess_text(keyword_query)
    similarity_score = model.predict([manual_abstract_seq, keyword_query_seq])[0][0]
    return similarity_score

In [96]:
# Example usage
new_manual_abstract = "The use of artificial intelligence and machine learning is growing rapidly, especially in natural language understanding applications."
new_keyword_query = "artificial intelligence OR natural language understanding"

similarity_score = test_similarity(new_manual_abstract, new_keyword_query)
print(f"Similarity score: {similarity_score:.4f}")

1/1 [==============================] - 1s 879ms/step
Similarity score: 0.9957
